In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from scipy.signal import find_peaks

In [2]:
#Importing the CSV file
SYMBOL = 'eurusd'.upper()
path = f"./OHLC_data/{SYMBOL}_20240715_20240803.csv"

OHLC_df = pd.read_csv(path)

In [3]:
#OHLC_df['datetime'] = pd.to_datetime(OHLC_df['date'].astype('str') + ' ' + OHLC_df['time'].astype('str'), format='%d/%m/%Y %H:%M')
#OHLC_df['datetime'] = pd.to_datetime(OHLC_df['date'] + ' ' + OHLC_df['time'], format='%d/%m/%Y %H:%M')
#OHLC_df['datetime'] = pd.to_datetime(OHLC_df['date'] + ' ' + OHLC_df['time'])
OHLC_df['datetime'] = pd.to_datetime(OHLC_df['date'] + ' ' + OHLC_df['time'], dayfirst=True)
#OHLC_df['datetime'] = pd.to_datetime(OHLC_df['date'] + ' ' + OHLC_df['time'], format='%Y-%m-%d %H:%M')

In [4]:
#Functions
            #Pa2 = p1_range_localmaximas[condition5 & condition ].drop(['open','low','close'], axis=1)
            #Pa2 = Pa2.drop_duplicates(subset=['high'], keep='last')
            #Pa2 = Pa2.rename(columns={"high":"price"})
            #HH_HL_df = pd.concat([HH_HL_df, Pa2], ignore_index=True)

def rename_column_price_n(Pay):
    Pa = p1_range_localminimas[p1_range_localminimas['low'] == Pay].drop(['open','high','close'], axis=1)
    Pa = Pa.drop_duplicates(subset=['low'], keep='last')
    Pa = Pa.rename(columns={"low":"price"})
    return Pa

def rename_column_price_x(Pay, condition):
    Pa = p1_range_localmaximas[(p1_range_localmaximas['high'] == Pay) & condition].drop(['open','low','close'], axis=1)
    Pa = Pa.drop_duplicates(subset=['high'], keep='last')
    Pa = Pa.rename(columns={"high":"price"})
    return Pa

def compute_index_Pa_n(Pay):
    Pay_index = p1_range_localminimas[p1_range_localminimas['low'] == Pay].index[-1]
    return Pay_index

def create_tuples_p1_p3(df):
    p1_p2_p3 = []
    max_index = df.index.max()
    no_of_comb = ((max_index + 1) // 2) -1  # +1 to include the last element for odd lengths
    for i in range(no_of_comb):
        p1_p2_p3.append((max_index, max_index - 2*(i+1) + 1, max_index - 2*(i+1)))
    return p1_p2_p3 

In [5]:
OHLC_df.drop(["date","time"], axis=1)

open     high      low    close            datetime
0      1.08860  1.08860  1.08842  1.08842 2024-07-15 00:00:00
1      1.08842  1.08845  1.08841  1.08841 2024-07-15 00:01:00
2      1.08841  1.08841  1.08841  1.08841 2024-07-15 00:02:00
3      1.08840  1.08853  1.08818  1.08827 2024-07-15 00:03:00
4      1.08827  1.08830  1.08818  1.08821 2024-07-15 00:04:00
...        ...      ...      ...      ...                 ...
21109  1.08958  1.08969  1.08943  1.08952 2024-08-02 15:56:00
21110  1.08950  1.08950  1.08884  1.08888 2024-08-02 15:57:00
21111  1.08888  1.08902  1.08878  1.08902 2024-08-02 15:58:00
21112  1.08902  1.08918  1.08886  1.08912 2024-08-02 15:59:00
21113  1.08904  1.08904  1.08856  1.08864 2024-08-02 16:00:00

[21114 rows x 5 columns]

In [6]:
OHLC_df = OHLC_df.reindex(labels=["datetime","open","high","low","close"], axis=1)

In [7]:
#obtaining the OHLC data
start_dt="2024-07-31 15:00:00"
end_dt="2024-07-31 19:00:00"

rates_df = OHLC_df[(OHLC_df['datetime'] >= start_dt) & (OHLC_df['datetime'] <= end_dt)]

#OBTAINING THE HIGHEST POINT DURING THE NY SESSION
p3y = rates_df['high'].max()

#OBTAINING THE INDEX OF THE HIGHEST POINT
p3i = (rates_df[rates_df['high']== p3y])

#obtaining the OHLC data of P3
p3 = p3i.iloc[[-1]]

#converting the datatype of the datetime column
rates_df.loc[:,'datetime'] = pd.to_datetime(rates_df["datetime"])

#obtaining the data within the time limit 4 to 45 mins before P3
p1x_range_1 = p3["datetime"].iloc[0] - timedelta(minutes = 45)
p1x_range_2 = p3["datetime"].iloc[0] - timedelta(minutes = 0)
mask1= rates_df["datetime"].ge(p1x_range_1) & rates_df["datetime"].le(p1x_range_2)
p1_range = rates_df[mask1].reset_index(drop=True)

In [8]:
# Find the index of local maximas
local_maximas_i,_ = find_peaks(p1_range['high'].values)
local_maximas_i = np.array(local_maximas_i)

# Find the index of the local minima by inverting the data
local_minimas_i,_ = find_peaks(-p1_range['low'].values)
local_minimas_i = np.array(local_minimas_i)

#Obtaining the dataframe of the local minimas and local maximas
p1_range_localminimas = p1_range[p1_range.index.isin(local_minimas_i)].reset_index(drop=True)
p1_range_localmaximas = p1_range[p1_range.index.isin(local_maximas_i)].reset_index(drop=True)

In [9]:
#Finding the value of Pay1
Pay1 = p1_range_localminimas["low"].min()

#Finding the index of the first local minima
Pay1_index = p1_range_localminimas[p1_range_localminimas['low'] == Pay1].index.min()


#Dropping the minimas before the first local minimas
mask = p1_range_localminimas.index >= Pay1_index
p1_range_localminimas = p1_range_localminimas[mask].reset_index(drop=True)
p1_range_localminimas
#Finding the index of the first local minima
Pay1_index = compute_index_Pa_n(Pay1)

In [10]:
#to find the y value of the possible 2nd local minima:
Pay3 = p1_range_localminimas.iloc[Pay1_index+1:,3].min()
Pay3_index = compute_index_Pa_n(Pay3)

In [11]:
while True:
    try:
        #finding the index of the 1st local maxima
        condition1 = p1_range_localmaximas["datetime"].gt(p1_range_localminimas.iloc[Pay1_index,0])
        condition2 = p1_range_localmaximas["datetime"].lt(p1_range_localminimas.iloc[Pay3_index,0])
        condition = condition1 & condition2
        #Test value
        #condition = pd.Series([False] * 7)
        
        #to find the y value of the 1st local maxima:
        Pay2 = p1_range_localmaximas[condition == True]['high'].values
        if len(Pay2) == 0:
            #proceed with the next pay3 index and rerun the while loop
            Pay3_index = Pay3_index+1
            Pay3 = p1_range_localminimas.iloc[Pay3_index:,3].min()
            print(Pay3)
            continue
        else:
            Pay2=Pay2.max()
            break
    except IndexError as err1:
        break

In [12]:
#Making a new dataframe of the higher highs and higher lows approaching P3 in chronological order
HH_HL_df = pd.DataFrame(columns=['datetime','price'])
HH_HL_df = pd.DataFrame({'datetime': pd.Series(dtype='datetime64[ns]'), 'price': pd.Series(dtype='float')})

#Grabbing the data attached to Pay*
HH_HL_df = pd.concat([HH_HL_df, rename_column_price_n(Pay1)], ignore_index=True)

Pa2 = p1_range_localmaximas[p1_range_localmaximas['high'] == Pay2].drop(['open','low','close'], axis=1)
Pa2 = Pa2.drop_duplicates(subset=['high'], keep='last')
Pa2 = Pa2.rename(columns={"high":"price"})
HH_HL_df = pd.concat([HH_HL_df, Pa2], ignore_index=True)

HH_HL_df = pd.concat([HH_HL_df, rename_column_price_n(Pay3)], ignore_index=True)

In [13]:
#Resetting the values to avoid issues
Pay2 = Pay1_index = Pay3_index = 0

#Assigning new values to find new sets of Pa2, Pa3
Pay1 = Pay3
Pay1_index = compute_index_Pa_n(Pay1)

#Resetting the values to avoid issues
Pay3 = Pay3_index = 0

In [14]:
#Finding the initial value of Pay3
Pay3 = p1_range_localminimas.iloc[Pay1_index+1:,3].min()
Pay3_index = compute_index_Pa_n(Pay3)


#Finding the values of the higher lows and higher highs and appending it to the HH_HL_df
while True:
    Pay3_index_range = 0
    try:
        # Check if the current Pay3_index is beyond the end of p1_range_localminimas
        if Pay3_index > len(p1_range_localminimas):
            print(f"End of p1_range_localminimas reached.{Pay3_index}")
            break
            
        #finding the index of the next local maxima
        condition1 = p1_range_localmaximas["datetime"].gt(p1_range_localminimas.iloc[Pay1_index,0])
        condition2 = p1_range_localmaximas["datetime"].le(p1_range_localminimas.iloc[Pay3_index,0])
        condition = condition1 & condition2

        
        #to find the y value of the next local maxima:
        Pay2 = p1_range_localmaximas[condition == True]['high'].values
        
        #previous HH index
        pHH_index = HH_HL_df[HH_HL_df['price'] == Pay1].index[0] - 1
        pHHy = HH_HL_df.iloc[pHH_index,1]

        #if Pay2 isnt existing for the current Pay3
        condition3 = len(Pay2) == 0
        
        #if the current Pay2 is less than the previous HH
        try:
            condition4 = p1_range_localmaximas[condition == True]['high'].values.max() < pHHy
        except ValueError as err2:
            Pay3_index = Pay3_index+1
            Pay3 = p1_range_localminimas.iloc[Pay3_index:,3].min()
            Pay3_index = compute_index_Pa_n(Pay3)
            continue
        if condition3 or condition4:
            #proceed with the next pay3 index and rerun the while loop
            Pay2 = 0
            Pay3_index_range = Pay3_index+1
            Pay3 = p1_range_localminimas.iloc[Pay3_index_range:,3].min()
            Pay3_index = compute_index_Pa_n(Pay3)
            continue
        else:
            Pay2=Pay2.max()
            #Appending to the HH_HL_df 
            condition5 = (p1_range_localmaximas['high'] == Pay2) & condition
            HH_HL_df = pd.concat([HH_HL_df, rename_column_price_x(Pay2, condition5)], ignore_index=True)
            HH_HL_df = pd.concat([HH_HL_df, rename_column_price_n(Pay3)], ignore_index=True)
            
            #Assigning new values to find new sets of Pa2, Pa3
            Pay1 = Pay3
            Pay1_index = compute_index_Pa_n(Pay1)
            
            #Finding the initial value of Pay3
            Pay3 = p1_range_localminimas.iloc[Pay1_index+1:,3].min()
            Pay3_index = compute_index_Pa_n(Pay3)
            continue
    except IndexError as err1:
        break

In [15]:
#Append p3 to the dataframe:
p3 = p3.drop(['open','low','close'], axis=1)
p3 = p3.rename(columns={"high":"price"})
HH_HL_df = pd.concat([HH_HL_df, p3], ignore_index=True)

In [16]:
# Create a tuple combining the list of possible p1-p2-p3i
p1_p2_p3i = create_tuples_p1_p3(HH_HL_df)
p1_p2_p3i

[(7, 6, 5), (7, 4, 3), (7, 2, 1)]

In [17]:
p1_p2_p3_index = []
for i in p1_p2_p3i:
    p3i=i[0]
    p2i=i[1]
    p1i=i[2]
    p3x_p2x = HH_HL_df.iloc[p3i,1] - HH_HL_df.iloc[p2i,1]
    p2x_p1x = HH_HL_df.iloc[p2i,1] - HH_HL_df.iloc[p1i,1]
    p3y_p2y = HH_HL_df.iloc[p3i,0] - HH_HL_df.iloc[p2i,0]
    p2y_p1y = HH_HL_df.iloc[p2i,0] - HH_HL_df.iloc[p1i,0]
    
    if (1.027 < abs(p3x_p2x/p2x_p1x) < 2.337) and (0.333 < p3y_p2y/p2y_p1y < 5):
        p1_p2_p3_index.append(i)
        print(i)

(7, 6, 5)
(7, 4, 3)


# Values for Testing Purposes

In [19]:
#previous HH index
HH_HL_df.head(10)

datetime    price
0 2024-07-31 15:57:00  1.08368
1 2024-07-31 16:00:00  1.08425
2 2024-07-31 16:06:00  1.08386
3 2024-07-31 16:23:00  1.08464
4 2024-07-31 16:26:00  1.08412
5 2024-07-31 16:31:00  1.08478
6 2024-07-31 16:34:00  1.08448
7 2024-07-31 16:36:00  1.08493

In [20]:
p1_range_localminimas

datetime     open     high      low    close
0 2024-07-31 15:57:00  1.08379  1.08381  1.08368  1.08380
1 2024-07-31 16:02:00  1.08397  1.08397  1.08386  1.08393
2 2024-07-31 16:06:00  1.08402  1.08402  1.08386  1.08394
3 2024-07-31 16:14:00  1.08437  1.08447  1.08426  1.08431
4 2024-07-31 16:17:00  1.08437  1.08437  1.08425  1.08430
5 2024-07-31 16:21:00  1.08449  1.08454  1.08436  1.08442
6 2024-07-31 16:26:00  1.08415  1.08422  1.08412  1.08420
7 2024-07-31 16:31:00  1.08461  1.08478  1.08456  1.08477
8 2024-07-31 16:34:00  1.08461  1.08461  1.08448  1.08456

In [21]:
p1_range_localmaximas

datetime     open     high      low    close
0 2024-07-31 15:52:00  1.08414  1.08425  1.08412  1.08414
1 2024-07-31 15:55:00  1.08408  1.08416  1.08403  1.08409
2 2024-07-31 16:00:00  1.08408  1.08425  1.08408  1.08417
3 2024-07-31 16:03:00  1.08394  1.08412  1.08394  1.08397
4 2024-07-31 16:05:00  1.08400  1.08404  1.08391  1.08401
5 2024-07-31 16:12:00  1.08446  1.08453  1.08441  1.08442
6 2024-07-31 16:14:00  1.08437  1.08447  1.08426  1.08431
7 2024-07-31 16:19:00  1.08453  1.08456  1.08446  1.08454
8 2024-07-31 16:23:00  1.08459  1.08464  1.08451  1.08456
9 2024-07-31 16:31:00  1.08461  1.08478  1.08456  1.08477